# Processamento de Dados

Este notebook encontra-se dividido em três partes. A primeira parte contém todo o processamento de dados efectuado aos dados recolhidos para a construção do dataset daily_covid, e asegunda contém todos os passos aplicados na construção do dataset daily_diabetes.


Para o processamento dos diferentes datasets foram utilizadas as bibliotecas python: numpy e pandas

In [1]:
import numpy as np
import pandas as pd

Devido aos diferentes sinais separadores de atributos dos registos presentes nos datasets utilizados, foi necessário criar funções diferentes para o load dos datasets.

In [2]:
def load_normal_dataset(path):
    return pd.read_csv(path)

In [3]:
def load_datasets(path):
    return pd.read_csv(path, sep=';')

O método to_csv_file, é utilizado para construir o dataset final que posteriormente será utilizado nas previsões.

In [4]:
def to_csv_file(df, file_name):
    df.to_csv(file_name)

## Parte 1 - Daily_covid dataset

Nesta Parte, irei pormenorizar todos as escolhas e passos tomados, para que fosse possível construir um dataset final, capaz de ser robusto o suficiente para eventualmente se obter uma boa previsão para o número de mortes causadas por covid-19 em Portugal. Para a construção deste dataset foram utilizados dados obtidos no repositório [Data Science for Social Good Portugal(DDSGP)](https://github.com/dssg-pt/covid19pt-data), que correspondem a registos diários de mortes por covid e dados atmosféricos obtidos através da plataforma [Visualcrossing](https://www.visualcrossing.com/weather/weather-data-services#/login). Estes dados atmosféricos são as médias diárias para Portugal no mesmo periodo de registos observados no dataset do repositório DSSGP.

### Load do dataset daily_covid_19_portugal

In [5]:
df_raw_covid = load_normal_dataset("daily_datasets/daily_covid_19_portugal.csv")

### Load dos datasets atmosféricos de Portugal

In [6]:
df_raw_temp_2020_01 = load_normal_dataset('daily_datasets/temp_portugal_01012020_31032020.csv')
df_raw_temp_2020_02 = load_normal_dataset('daily_datasets/temp_portugal_01042020_30062020.csv')
df_raw_temp_2020_03 = load_normal_dataset('daily_datasets/temp_portugal_01072020_30092020.csv')
df_raw_temp_2020_04 = load_normal_dataset('daily_datasets/temp_portugal_01102020_31122020.csv')
df_raw_temp_2020_05 = load_normal_dataset('daily_datasets/temp_portugal_01012021_31032021.csv')
df_raw_temp_2020_06 = load_normal_dataset('daily_datasets/temp_portugal_042021.csv')

Após estes loads, foi necessário efetuar tratamento de dados obtidos, nomeadamente tratamento de missing values, alterações dos tipos dos atributos e por fim a concatenação dos diferentes datasets num só dataset.

### Tratamento dos datasets

Em prepare_data_covid, encontra-se todo o tratamento efectuado ao dataset daily_covid_19_portugal. Nesta função foram eliminados atributos cujas colunas apresentavam número de missing válues superiores a 20% da totalidade dos dados. Eliminamos a diferenciação do número de óbitos e infectados por género, para isso somou-se o valor dos atributos correspondentes à mesma idade, resultando em apenas uma coluna correspondente à sua respetiva idade (ex: obitos_20_29m + obitos_20_29f = obitos_20_29, infetados_20_29m + infetados_20_29f = infetados_20_29). Atributos cujo o número de missing values era inferior a 20%, esses missing values foram substituídos pelo valor registado no dia anterior (para missing values referentes a registos não iniciais da série temporal), ou foram substituídos pelo valor 0 (para missing values referentes a registos no periodo inicial da pandemia)

In [9]:
def prepare_data_covid(df_raw_covid):
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    df_aux = df_raw_covid.copy()
    print(df_aux.shape)

    print("Number of missing Values: ")
    print(df_aux.isnull().sum())


    df_aux = df_raw_covid.drop(columns=['data_dados', 'confirmados_estrangeiro', 'lab', 'suspeitos', 'vigilancia',
                                        'n_confirmados', 'cadeias_transmissao', 'transmissao_importada',
                                        'sintomas_tosse', 'sintomas_febre', 'sintomas_cefaleia',
                                        'sintomas_dores_musculares', 'sintomas_dificuldade_respiratoria', 'sintomas_fraqueza_generalizada',
                                        'obitos_estrangeiro', 'recuperados_arsnorte','recuperados_arscentro',
                                        'recuperados_arslvt','recuperados_arsalentejo',
                                        'recuperados_arsalgarve','recuperados_acores', 'recuperados_madeira',
                                        'recuperados_estrangeiro','confirmados_desconhecidos_m','confirmados_desconhecidos_f',
                                        'incidencia_nacional','incidencia_continente','rt_nacional', 'rt_continente',
                                        'obitos_f', 'obitos_m','confirmados_desconhecidos','confirmados_f','confirmados_m',
                                        'internados'], inplace=False)


    #somar colunas casos confirmados com mesma faixa etaria ignorando sexo
    df_aux['confirmados_0_9'] = df_aux.apply(lambda x: x['confirmados_0_9_f'] + x['confirmados_0_9_m'], axis=1)
    df_aux['confirmados_10_19'] = df_aux.apply(lambda x: x['confirmados_10_19_f'] + x['confirmados_10_19_m'], axis=1)
    df_aux['confirmados_20_29'] = df_aux.apply(lambda x: x['confirmados_20_29_f'] + x['confirmados_20_29_m'], axis=1)
    df_aux['confirmados_30_39'] = df_aux.apply(lambda x: x['confirmados_30_39_f'] + x['confirmados_30_39_m'], axis=1)
    df_aux['confirmados_40_49'] = df_aux.apply(lambda x: x['confirmados_40_49_f'] + x['confirmados_40_49_m'], axis=1)
    df_aux['confirmados_50_59'] = df_aux.apply(lambda x: x['confirmados_50_59_f'] + x['confirmados_50_59_m'], axis=1)
    df_aux['confirmados_60_69'] = df_aux.apply(lambda x: x['confirmados_60_69_f'] + x['confirmados_60_69_m'], axis=1)
    df_aux['confirmados_70_79'] = df_aux.apply(lambda x: x['confirmados_70_79_f'] + x['confirmados_70_79_m'], axis=1)
    df_aux['confirmados_80_plus'] = df_aux.apply(lambda x: x['confirmados_80_plus_f'] + x['confirmados_80_plus_m'], axis=1)

    df_aux = df_aux.drop(columns=['confirmados_0_9_f','confirmados_0_9_m',
                            'confirmados_10_19_f','confirmados_10_19_m',
                            'confirmados_20_29_f','confirmados_20_29_m',
                            'confirmados_30_39_f','confirmados_30_39_m',
                            'confirmados_40_49_f','confirmados_40_49_m',
                            'confirmados_50_59_f','confirmados_50_59_m',
                            'confirmados_60_69_f','confirmados_60_69_m',
                            'confirmados_70_79_f','confirmados_70_79_m',
                            'confirmados_80_plus_f','confirmados_80_plus_m'], inplace=False)


    #somar colunas obitos confirmados com mesma faixa etaria ignorando sexo
    df_aux['obitos_0_9'] = df_aux.apply(lambda x: x['obitos_0_9_f'] + x['obitos_0_9_m'], axis=1)
    df_aux['obitos_10_19'] = df_aux.apply(lambda x: x['obitos_10_19_f'] + x['obitos_10_19_m'], axis=1)
    df_aux['obitos_20_29'] = df_aux.apply(lambda x: x['obitos_20_29_f'] + x['obitos_20_29_m'], axis=1)
    df_aux['obitos_30_39'] = df_aux.apply(lambda x: x['obitos_30_39_f'] + x['obitos_30_39_m'], axis=1)
    df_aux['obitos_40_49'] = df_aux.apply(lambda x: x['obitos_40_49_f'] + x['obitos_40_49_m'], axis=1)
    df_aux['obitos_50_59'] = df_aux.apply(lambda x: x['obitos_50_59_f'] + x['obitos_50_59_m'], axis=1)
    df_aux['obitos_60_69'] = df_aux.apply(lambda x: x['obitos_60_69_f'] + x['obitos_60_69_m'], axis=1)
    df_aux['obitos_70_79'] = df_aux.apply(lambda x: x['obitos_70_79_f'] + x['obitos_70_79_m'], axis=1)
    df_aux['obitos_80_plus'] = df_aux.apply(lambda x: x['obitos_80_plus_f'] + x['obitos_80_plus_m'], axis=1)

    df_aux = df_aux.drop(columns=['obitos_0_9_f','obitos_0_9_m',
                            'obitos_10_19_f','obitos_10_19_m',
                            'obitos_20_29_f','obitos_20_29_m',
                            'obitos_30_39_f','obitos_30_39_m',
                            'obitos_40_49_f','obitos_40_49_m',
                            'obitos_50_59_f','obitos_50_59_m',
                            'obitos_60_69_f','obitos_60_69_m',
                            'obitos_70_79_f','obitos_70_79_m',
                            'obitos_80_plus_f','obitos_80_plus_m'], inplace=False)

    #drop das ultimas 2 rows do dataset
    df_aux = df_aux.drop(df_aux.tail(2).index)

    #fill row com missing values aplicando ffill method, n vejo problema pois aind a não haviam casos e faltando um dia mantem o valor do dia anterio
    #obitos_80_plus
    df_aux = df_aux.fillna(method='ffill', limit=1)
    #obitos_arsalentejo
    df_aux = df_aux.fillna(method='ffill', limit=19)
    #como os nan estão ao inicio da pandemia, é natural n terem valor, logo vou substituir esses NaN por 0
    df_aux = df_aux.fillna(0)

    #print(df_aux.isnull().sum())
    #print(df_aux)
    #print("Number of missing Values: ")
    #print(df_aux.isnull().sum())
    # counting the duplicates
    dups = df_aux.pivot_table(index=['data','confirmados'], aggfunc='size')
    # displaying the duplicate Series
    print(dups)
    #Não tem duplicados


    df_aux.columns = ['Date', 'confirmados', 'confirmados_arsnorte', 'confirmados_arscentro',
     'confirmados_arslvt', 'confirmados_arsalentejo',
     'confirmados_arsalgarve', 'confirmados_acores', 'confirmados_madeira',
     'confirmados_novos', 'recuperados', 'obitos', 'internados_uci',
     'obitos_arsnorte', 'obitos_arscentro', 'obitos_arslvt',
     'obitos_arsalentejo', 'obitos_arsalgarve', 'obitos_acores',
     'obitos_madeira', 'ativos', 'internados_enfermaria', 'confirmados_0_9',
     'confirmados_10_19', 'confirmados_20_29', 'confirmados_30_39',
     'confirmados_40_49', 'confirmados_50_59', 'confirmados_60_69',
     'confirmados_70_79', 'confirmados_80_plus', 'obitos_0_9',
     'obitos_10_19', 'obitos_20_29', 'obitos_30_39', 'obitos_40_49',
     'obitos_50_59', 'obitos_60_69', 'obitos_70_79', 'obitos_80_plus']

    #Colocar a diferença em vezz de valor acumulado nos dias.
    df_aux["confirmados"] = df_aux.loc[:,["confirmados"]].diff()
    df_aux["confirmados_arsnorte"] = df_aux.loc[:,["confirmados_arsnorte"]].diff()
    df_aux["confirmados_arscentro"] = df_aux.loc[:,["confirmados_arscentro"]].diff()
    df_aux["confirmados_arslvt"] = df_aux.loc[:,["confirmados_arslvt"]].diff()
    df_aux["confirmados_arsalentejo"] = df_aux.loc[:,["confirmados_arsalentejo"]].diff()
    df_aux["confirmados_arsalgarve"] = df_aux.loc[:,["confirmados_arsalgarve"]].diff()
    df_aux["confirmados_acores"] = df_aux.loc[:,["confirmados_acores"]].diff()
    df_aux["confirmados_madeira"] = df_aux.loc[:,["confirmados_madeira"]].diff()
    df_aux["recuperados"] = df_aux.loc[:,["recuperados"]].diff()
    df_aux["obitos"] = df_aux.loc[:,["obitos"]].diff()
    df_aux["internados_uci"] = df_aux.loc[:,["internados_uci"]].diff()
    df_aux["obitos_arsnorte"] = df_aux.loc[:,["obitos_arsnorte"]].diff()
    df_aux["obitos_arscentro"] = df_aux.loc[:,["obitos_arscentro"]].diff()
    df_aux["obitos_arslvt"] = df_aux.loc[:,["obitos_arslvt"]].diff()
    df_aux["obitos_arsalentejo"] = df_aux.loc[:,["obitos_arsalentejo"]].diff()
    df_aux["obitos_arsalgarve"] = df_aux.loc[:,["obitos_arsalgarve"]].diff()
    df_aux["obitos_acores"] = df_aux.loc[:,["obitos_acores"]].diff()
    df_aux["obitos_madeira"] = df_aux.loc[:,["obitos_madeira"]].diff()
    df_aux["ativos"] = df_aux.loc[:,["ativos"]].diff()
    df_aux["internados_enfermaria"] = df_aux.loc[:,["internados_enfermaria"]].diff()
    df_aux["confirmados_0_9"] = df_aux.loc[:,["confirmados_0_9"]].diff()
    df_aux["confirmados_10_19"] = df_aux.loc[:,["confirmados_10_19"]].diff()
    df_aux["confirmados_20_29"] = df_aux.loc[:,["confirmados_20_29"]].diff()
    df_aux["confirmados_30_39"] = df_aux.loc[:,["confirmados_30_39"]].diff()
    df_aux["confirmados_40_49"] = df_aux.loc[:,["confirmados_40_49"]].diff()
    df_aux["confirmados_50_59"] = df_aux.loc[:,["confirmados_50_59"]].diff()
    df_aux["confirmados_60_69"] = df_aux.loc[:,["confirmados_60_69"]].diff()
    df_aux["confirmados_70_79"] = df_aux.loc[:,["confirmados_70_79"]].diff()
    df_aux["confirmados_80_plus"] = df_aux.loc[:,["confirmados_80_plus"]].diff()
    df_aux["obitos_0_9"] = df_aux.loc[:,["obitos_0_9"]].diff()
    df_aux["obitos_10_19"] = df_aux.loc[:,["obitos_10_19"]].diff()
    df_aux["obitos_20_29"] = df_aux.loc[:,["obitos_20_29"]].diff()
    df_aux["obitos_30_39"] = df_aux.loc[:,["obitos_30_39"]].diff()
    df_aux["obitos_40_49"] = df_aux.loc[:,["obitos_40_49"]].diff()
    df_aux["obitos_50_59"] = df_aux.loc[:,["obitos_50_59"]].diff()
    df_aux["obitos_60_69"] = df_aux.loc[:,["obitos_60_69"]].diff()
    df_aux["obitos_70_79"] = df_aux.loc[:,["obitos_70_79"]].diff()
    df_aux["obitos_80_plus"] = df_aux.loc[:,["obitos_80_plus"]].diff()
    
    #pd.set_option('display.max_rows', None)
    #pd.set_option('display.max_columns', None)
    #print(df_aux.columns)
    #print(df_aux)

    return df_aux

In [10]:
df_data_covid = prepare_data_covid(df_raw_covid)

(424, 93)
Number of missing Values: 
data                                   0
data_dados                             0
confirmados                            0
confirmados_arsnorte                   0
confirmados_arscentro                  0
confirmados_arslvt                     0
confirmados_arsalentejo                0
confirmados_arsalgarve                 0
confirmados_acores                     0
confirmados_madeira                    0
confirmados_estrangeiro              408
confirmados_novos                      0
recuperados                            0
obitos                                 0
internados                             8
internados_uci                        17
lab                                  260
suspeitos                            251
vigilancia                             7
n_confirmados                        269
cadeias_transmissao                  409
transmissao_importada                257
confirmados_0_9_f                      8
confirmados_0_9_m   

Em append_tem_dataframes, procedeu-se à junção de todos os datasets referentes aos dados atmosféricos de Portugal, para ser possível um tratamento mais fácil desses datasets.

In [11]:
def append_temp_dataframes(df_raw_temp_2020_01, df_raw_temp_2020_02, df_raw_temp_2020_03, df_raw_temp_2020_04,
                           df_raw_temp_2020_05, df_raw_temp_2020_06):
    df_raw = df_raw_temp_2020_01.append(df_raw_temp_2020_02, ignore_index = True)
    df_raw = df_raw.append(df_raw_temp_2020_03, ignore_index = True)
    df_raw = df_raw.append(df_raw_temp_2020_04, ignore_index = True)
    df_raw = df_raw.append(df_raw_temp_2020_05, ignore_index = True)
    df_raw = df_raw.append(df_raw_temp_2020_06, ignore_index = True)

    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    #print(df_raw)

    return df_raw

In [12]:
df_raw_appended_temp = append_temp_dataframes(df_raw_temp_2020_01,df_raw_temp_2020_02,df_raw_temp_2020_03,df_raw_temp_2020_04,df_raw_temp_2020_05,df_raw_temp_2020_06)


Na função prepare_data_temp, encontra-se todo o processamento aplicado aos dados atmosféricos. Aqui, são eliminadas as coluna com missing values superiores a 80% da totalidade dos registos observados, e é executadado one-hot-encoding ao atributo Conditions através de aplicação da função apply().

In [13]:
def condition_ohe_rain(condition):
    res =0
    if(condition == 'Rain' or condition == 'Rain, Partially cloudy'):
        res=1

    return res




In [14]:
def condition_ohe_clear(condition):
    res =0
    if(condition == 'Clear'):
        res=1

    return res




In [15]:
def condition_ohe_cloudy(condition):
    res =0
    if(condition == 'Rain, Partially cloudy' or condition == 'Partially cloudy'):
        res=1

    return res

In [16]:
def prepare_data_temp(df_raw):

    df_aux = df_raw.copy()
    #Ver se tem missing values
    mv = df_aux.isnull().sum()
    print(mv)


    #drop de colunas com muitos missing values e a coluna nome do país, colunas referentes á neve(têm apenas valores ='s a 0)
    df_aux = df_aux.drop(columns=['Name','Wind Chill','Heat Index', 'Wind Gust', 'Snow', 'Snow Depth'])

    #rename columns
    df_aux.columns = ['Date', 'Max_Temp','Min_Temp','Temperature','Precipitation','Wind_Speed','Wind_Direction','Visibility','Cloud_Cover','Relative_Humidity','Conditions']

    #one hot encoding weather conditions
    df_aux['Rain'] = np.nan
    df_aux['Clear'] = np.nan
    df_aux['Partially_cloudy'] = np.nan
    
    df_aux['Rain'] = df_aux.apply(lambda x: condition_ohe_rain(x['Conditions']), axis=1)
    df_aux['Clear'] = df_aux.apply(lambda x: condition_ohe_clear(x['Conditions']), axis=1)
    df_aux['Partially_cloudy'] = df_aux.apply(lambda x: condition_ohe_cloudy(x['Conditions']), axis=1)

    df_aux = df_aux.drop(columns='Conditions')
    #colocar data igual ao dataset daily_covid_19_portugal
    df_aux["Date"] = pd.to_datetime(df_aux["Date"]).dt.strftime('%d-%m-%Y')

    return df_aux


In [17]:
df_data_temp = prepare_data_temp(df_raw_appended_temp)

Name                     0
Date time                0
Maximum Temperature      0
Minimum Temperature      0
Temperature              0
Wind Chill             400
Heat Index             399
Precipitation            0
Snow                     0
Snow Depth               0
Wind Speed               0
Wind Direction           0
Wind Gust              343
Visibility               0
Cloud Cover              0
Relative Humidity        0
Conditions               0
dtype: int64


Após o processamentodos diferentes datasets, procedeu-se à concatenação dos mesmos através da operação merge(df_data_covid,df_data_temp,on='Date',how='left'), uma vez os registos atmosféricos são de um periodo temporal superior aos registos temporais da pandemia.

In [18]:
def unifie_covid_datasets(df_data_covid, df_data_temp):

    df_covid = pd.merge(df_data_covid,df_data_temp, on='Date', how='left')

    print(df_data_covid.shape)
    print(df_data_temp.shape)

    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)

    return df_covid


In [19]:
df_covid = unifie_covid_datasets(df_data_covid, df_data_temp)

(422, 40)
(486, 13)


Por fim é necessário criar o dataset final, que será utilizaos nas previsões sobre as mortes por covid em Portugal.

In [20]:
df_covid["Date"] = pd.to_datetime(df_covid["Date"])
df_covid = df_covid.set_index('Date')
df_covid.head()

,confirmados,confirmados_arsnorte,confirmados_arscentro,confirmados_arslvt,confirmados_arsalentejo,confirmados_arsalgarve,confirmados_acores,confirmados_madeira,confirmados_novos,recuperados,obitos,internados_uci,obitos_arsnorte,obitos_arscentro,obitos_arslvt,obitos_arsalentejo,obitos_arsalgarve,obitos_acores,obitos_madeira,ativos,internados_enfermaria,confirmados_0_9,confirmados_10_19,confirmados_20_29,confirmados_30_39,confirmados_40_49,confirmados_50_59,confirmados_60_69,confirmados_70_79,confirmados_80_plus,obitos_0_9,obitos_10_19,obitos_20_29,obitos_30_39,obitos_40_49,obitos_50_59,obitos_60_69,obitos_70_79,obitos_80_plus,Max_Temp,Min_Temp,Temperature,Precipitation,Wind_Speed,Wind_Direction,Visibility,Cloud_Cover,Relative_Humidity,Rain,Clear,Partially_cloudy
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-02-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.4,10.0,12.9,0.10,22.3,325.25,12.3,7.5,68.15,1,0,0
2020-02-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.4,10.4,14.5,0.00,16.9,321.58,11.8,3.9,72.08,0,1,0
2020-02-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.1,10.3,13.3,0.00,17.7,123.33,10.1,3.8,79.20,0,1,0
2020-02-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,13.1,15.1,3.53,24.7,253.13,9.6,6.8,82.29,1,0,0
2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.1,14.0,15.3,0.68,38.6,235.63,9.5,4.3,92.37,1,0,0


In [21]:
to_csv_file(df_covid, "daily_covid.csv")

## Parte 2 - Daily_diabetes dataset

Aqui, são pormenorizas todas as escolhas e passos tomados, para que fosse possível construir um dataset final, capaz de ser robusto o suficiente para eventualmente se obter uma boa previsão para o número de mortes causadas por diabetes em Inglaterra. Para a construção deste dataset foram utilizados dados sobre a temperatura em Inglaterra através do [European Climate Assessment & Dataset](https://www.ecad.eu/dailydata/customquery.php), o registo diário do nível medio de Ozono através [UK Air - Department for environment Food & Rural Affairs](https://uk-air.defra.gov.uk/interactive-map), e o registo de mortes por diabetes no ano de 2014 através de [England Office for National Statistics](https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/causesofdeath/adhocs/005259dailydeathoccurrencesbyallcausesanddiabetesmellitusicd10e10toe14englandandwales2012to2014)

### Load do dataset Diabetes_2014_England

In [22]:
df_raw_diabetes = load_datasets("daily_datasets/Diabetes_2014_England.csv")

### Load dos datasets das temperaturas em Inglaterra

Para podermos utilizar temperaturas médias registadas em Inglaterra, foi necessário recorrer a várias estações meteorológicas espalhas poir Inglaterra, as estações escolhidas foram as estações de Blackpool, Durham, Nottingham e Bognor Regis.

In [23]:
df_raw_temp_bognor_regis = load_normal_dataset("daily_datasets/mean_temp_bognor_regis.csv")
df_raw_temp_blackpool = load_normal_dataset("daily_datasets/mean_temp_blackpool.csv")
df_raw_temp_durham = load_normal_dataset("daily_datasets/mean_temp_durham.csv")
df_raw_temp_nottingham = load_normal_dataset("daily_datasets/mean_temp_nottingham.csv")

### Load dos datasets dos níveis de Ozono em Ingalterra

Para podermos utilizar os níveis médios de ozono registadas em Inglaterra, foi necessário recorrer a várias estações meteorológicas espalhas poir Inglaterra, as estações escolhidas foram as estações de Hull (Freetown), Liverpool (Wirral Tranmere), London (Bloomsbury) e Norwich (Lakenfields).

In [24]:
df_raw_ozone_hull = load_normal_dataset("daily_datasets/hull_freetown_ozone.csv")
df_raw_ozone_norwich = load_normal_dataset("daily_datasets/norwich_lakenfields_ozone.csv")
df_raw_ozone_wirral = load_normal_dataset("daily_datasets/wirral_tranmere_ozone.csv")
df_raw_ozone_london = load_normal_dataset("daily_datasets/london_ozone.csv")

Após estes loads, foi necessário efetuar tratamento de dados obtidos, nomeadamente tratamento de missing values, alterações dos tipos dos atributos, cálculo de média de medidas de ozono e temperatura e por fim a concatenação dos diferentes datasets num só dataset.

### Tratamento dos datasets

Em prepare_data_diabetes(), processamos os dados relativos ao número de mortes por diabetes em Inglaterra em 2014. O processamento foi mínimo, apenas os atributos foram renomeados e os seus tipos foram alterado como sendo numéricos numérico

In [25]:
def prepare_data_diabetes(df_raw):

    df_aux = df_raw.copy()

    df_aux.columns = ['Date','All_Causes','Diabetes']
    #print(df_aux)
    
    #tirar o espaço da string All_Causes
    df_aux['All_Causes'] = df_aux['All_Causes'].str.replace(" ",'')

    # alterar tipo das colunas All_Causes, Diabetes para numerico :
    # Alterar tipo da coluna Date para datetime
    df_aux["Diabetes"] = pd.to_numeric(df_aux["Diabetes"])
    df_aux['All_Causes'] = pd.to_numeric(df_aux['All_Causes'])

    #Ver se tem duplicados
    r  = df_aux.duplicated(subset='Date').sum()
    print(r)

    #Ver se tem missing values
    mv = df_aux.isnull().sum()
    print(mv)

    #somar mortes no mesmo dia
    #df_aux = df_aux.groupby(['Date']).sum()

    pd.set_option('display.max_rows', None)
    #print(df_aux)
    return df_aux

In [26]:
f_data_diabetes = prepare_data_diabetes(df_raw_diabetes)

0
Date          0
All_Causes    0
Diabetes      0
dtype: int64


Para o tratamentos dos datasets que contêm as temperaturas diárias registadas em Inglaterra, para cada um destes datasets foi necessário transformar a temperatura registada num valor real, atravéss da sua multiplicação por 0.1. Também foi necessário transformar a data do registo num formato válido, para depois ser possível concatenar os vários datasets pela sua data.

In [27]:
def prepare_data_temperature_england(df_raw,name):

    df_aux = df_raw.copy()
    print(df_aux.dtypes)
    print (df_aux.head())
    #pd.set_option('display.max_rows', None)
    #print(df_aux)
    #unique values of Station id (só tem 1 estaçao)
    print (len(df_aux['STAID'].unique()))
    #unique values od date (não tem duplicados)
    print(len(df_aux['DATE'].unique()))

    #drop colunas STAID,Q_TG
    df_aux = df_aux.drop(columns=['SOUID','STAID','Q_TG'])

    #tornar valores da temperatura, em valores reais   temperatura*0.1
    df_aux['TG'] = df_aux['TG']*0.1

    #converter integer date para a data correta: 19600101 para 1960-01-01
    df_aux['DATE'] = pd.to_datetime(df_aux['DATE'], format='%Y%m%d')

    #modificar data de 1960-01-01 para 01/01/1960
    df_aux["DATE"] = pd.to_datetime(df_aux["DATE"]).dt.strftime('%d/%m/%Y')

    nome ='temp_'+name
    #renomear colunas
    df_aux.columns = ['Date', nome]
    #print(df_aux.head())
    #print(df_aux.dtypes)
    return df_aux

In [28]:
f_data_temp_bognor_regis = prepare_data_temperature_england(df_raw_temp_bognor_regis, 'BOGNOR')
f_data_temp_blackpool = prepare_data_temperature_england(df_raw_temp_blackpool, 'BLACKP')
f_data_temp_durham = prepare_data_temperature_england(df_raw_temp_durham, 'DURHAM')
f_data_temp_nottingham = prepare_data_temperature_england(df_raw_temp_nottingham, 'NOTTI')

STAID    int64
SOUID    int64
DATE     int64
TG       int64
Q_TG     int64
dtype: object
   STAID   SOUID      DATE   TG  Q_TG
0   1863  107057  19600101  106     0
1   1863  107057  19600102   86     0
2   1863  107057  19600103   89     0
3   1863  107057  19600104   97     0
4   1863  107057  19600105  102     0
1
22371
STAID    int64
SOUID    int64
DATE     int64
TG       int64
Q_TG     int64
dtype: object
   STAID   SOUID      DATE  TG  Q_TG
0  11102  156831  20130101  66     0
1  11102  156831  20130102  79     0
2  11102  156831  20130103  78     0
3  11102  156831  20130104  86     0
4  11102  156831  20130105  71     0
1
3012
STAID    int64
SOUID    int64
DATE     int64
TG       int64
Q_TG     int64
dtype: object
   STAID   SOUID      DATE    TG  Q_TG
0    461  156851  18800101 -9999     9
1    461  156851  18800102    80     0
2    461  156851  18800103    85     1
3    461  156851  18800104    66     0
4    461  156851  18800105    50     0
1
51590
STAID    int64
SOUID    in

Após os tratementos dos diferentes datasets da temperatura em Inglaterra, e para utilizar-mos uma temperatura que representase Inglaterra como um todos foi necessário calcular a temperatura média para Inglaterra. Essa temperatura foi obtida através do método unifie_temp_mean()

In [29]:
def unifie_temp_mean(f_data_temp_bognor_regis,f_data_temp_blackpool,f_data_temp_durham,f_data_temp_nottingham):
    df_diabetes = pd.merge(f_data_temp_bognor_regis,f_data_temp_blackpool, on='Date', how='inner')
    df_diabetes = pd.merge(df_diabetes, f_data_temp_durham, on='Date', how='inner')
    df_diabetes = pd.merge(df_diabetes, f_data_temp_nottingham, on='Date', how='inner')
    #calculo da temperatura media
    df_diabetes['Temperature'] = df_diabetes.iloc[:, 1:4].mean(axis=1)
    # drop das colunas de temperatura das cidades
    cols= [1,2,3,4]
    df_diabetes = df_diabetes.drop(df_diabetes.columns[cols], axis=1)
    #print(df_diabetes.shape)
    #print(df_diabetes.head())
    return df_diabetes

In [30]:
df_temp_england = unifie_temp_mean(f_data_temp_bognor_regis, f_data_temp_blackpool, f_data_temp_durham, f_data_temp_nottingham)

Foi também necessário efetuar tratamento dos dados constantes nos datasets provenientes de diferentes centros de observação do nível de Ozono em Inglaterra. No método prepare_data_ozone_england(), uma vez que os registos eram diários e eram efectuadas medições do Ozono de hora a hora, os missing values foram substituidos pela média diária, e a data diária do registo foi colocada para um formato que fosse depois possível juntar com os outros datasets.

In [31]:
def prepare_data_ozone_england(df_raw,name):
    df_aux = df_raw.copy()

    #Ver se tem missing values
    mv = df_aux.isnull().sum()
    print('Estacao: ',name,' nº: ',mv)
    #print(df_aux.columns)
    #substituir missing values pelo valor médio da hora
    mean1,mean2,mean3,mean4,mean5,mean6,mean7,mean8,mean9,mean10,mean11,mean12 = df_aux['01:00'].mean(),df_aux['02:00'].mean(),df_aux['03:00'].mean(),df_aux['04:00'].mean(),df_aux['05:00'].mean(), df_aux['06:00'].mean(),df_aux['07:00'].mean(),df_aux['08:00'].mean(),df_aux['09:00'].mean(),df_aux['10:00'].mean(),df_aux['11:00'].mean(),df_aux['12:00'].mean()
    mean13, mean14, mean15, mean16, mean17, mean18, mean19, mean20, mean21, mean22, mean23, mean24 =  df_aux['13:00'].mean(),df_aux['14:00'].mean(),df_aux['15:00'].mean(),df_aux['16:00'].mean(),df_aux['17:00'].mean(),df_aux['18:00'].mean(),df_aux['19:00'].mean(),df_aux['20:00'].mean(),df_aux['21:00'].mean(),df_aux['22:00'].mean(),df_aux['23:00'].mean(),df_aux['24:00'].mean()


    df_aux['01:00'] = df_aux['01:00'].fillna(mean1)
    df_aux['02:00'] = df_aux['02:00'].fillna(mean2)
    df_aux['03:00'] = df_aux['03:00'].fillna(mean3)
    df_aux['04:00'] = df_aux['04:00'].fillna(mean4)
    df_aux['05:00'] = df_aux['05:00'].fillna(mean5)
    df_aux['06:00'] = df_aux['06:00'].fillna(mean6)
    df_aux['07:00'] = df_aux['07:00'].fillna(mean7)
    df_aux['08:00'] = df_aux['08:00'].fillna(mean8)
    df_aux['09:00'] = df_aux['09:00'].fillna(mean9)
    df_aux['10:00'] = df_aux['10:00'].fillna(mean10)
    df_aux['11:00'] = df_aux['11:00'].fillna(mean11)
    df_aux['12:00'] = df_aux['12:00'].fillna(mean12)
    df_aux['13:00'] = df_aux['13:00'].fillna(mean13)
    df_aux['14:00'] = df_aux['14:00'].fillna(mean14)
    df_aux['15:00'] = df_aux['15:00'].fillna(mean15)
    df_aux['16:00'] = df_aux['16:00'].fillna(mean16)
    df_aux['17:00'] = df_aux['17:00'].fillna(mean17)
    df_aux['18:00'] = df_aux['18:00'].fillna(mean18)
    df_aux['19:00'] = df_aux['19:00'].fillna(mean19)
    df_aux['20:00'] = df_aux['20:00'].fillna(mean20)
    df_aux['21:00'] = df_aux['21:00'].fillna(mean21)
    df_aux['22:00'] = df_aux['22:00'].fillna(mean22)
    df_aux['23:00'] = df_aux['23:00'].fillna(mean23)
    df_aux['24:00'] = df_aux['24:00'].fillna(mean24)

    # Ver se tem missing values
    #mv = df_aux.isnull().sum()
    #print('Estacao: ', name, ' nº: ', mv)

    #coluna media diária de ozono
    nome = 'ozone'+ name
    df_aux[nome] = df_aux.iloc[:, 1:25].mean(axis=1)

    #drop colunas das horas
    cols = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
    df_aux = df_aux.drop(df_aux.columns[cols], axis=1)

    #modificar data de 1960-01-01 para 01/01/1960
    df_aux["Date"] = pd.to_datetime(df_aux["Date"]).dt.strftime('%d/%m/%Y')

    return df_aux

In [32]:
f_data_ozone_hull = prepare_data_ozone_england(df_raw_ozone_hull, 'HULL')
f_data_ozone_norwich = prepare_data_ozone_england(df_raw_ozone_norwich, 'NORWICH')
f_data_ozone_wirral = prepare_data_ozone_england(df_raw_ozone_wirral, 'WINRRAL')
f_data_ozone_london = prepare_data_ozone_england(df_raw_ozone_london, 'LONDON')

Estacao:  HULL  nº:  Date      0
01:00    16
02:00    13
03:00    12
04:00    12
05:00    12
06:00    12
07:00    12
08:00    13
09:00    17
10:00    17
11:00    14
12:00    13
13:00    15
14:00    15
15:00    13
16:00    12
17:00    12
18:00    12
19:00    12
20:00    12
21:00    12
22:00    13
23:00    12
24:00    13
dtype: int64
Estacao:  NORWICH  nº:  Date     0
01:00    2
02:00    2
03:00    2
04:00    2
05:00    2
06:00    2
07:00    2
08:00    2
09:00    4
10:00    4
11:00    3
12:00    6
13:00    6
14:00    4
15:00    3
16:00    2
17:00    3
18:00    2
19:00    2
20:00    2
21:00    2
22:00    2
23:00    2
24:00    2
dtype: int64
Estacao:  WINRRAL  nº:  Date     0
01:00    3
02:00    2
03:00    2
04:00    2
05:00    2
06:00    2
07:00    2
08:00    2
09:00    2
10:00    3
11:00    4
12:00    7
13:00    7
14:00    7
15:00    7
16:00    6
17:00    3
18:00    3
19:00    3
20:00    3
21:00    3
22:00    3
23:00    3
24:00    3
dtype: int64
Estacao:  LONDON  nº:  Date      0
01:00  

Após o tratamento dos diferentes datasets do Ozono, procedeu-se á sua concatenação, formando apenas um dataframe com o registo diário de medição do nível médio de Ozono em Inglaterra , obtido através do cálculo da media dos valores registados nas diferentes estações de medição. 

In [33]:
def unifie_ozone_mean(f_data_ozone_hull,f_data_ozone_norwich,f_data_ozone_wirral,f_data_ozone_london):
    df_ozone = pd.merge(f_data_ozone_hull,f_data_ozone_norwich, on='Date', how='inner')
    df_ozone = pd.merge(df_ozone,f_data_ozone_wirral, on='Date', how='inner')
    df_ozone = pd.merge(df_ozone, f_data_ozone_london, on='Date', how='inner')
    #print(df_ozone.shape)
    #drop das colunas do ozone das cidades
    df_ozone['Ozone'] = df_ozone.iloc[:, 1:4].mean(axis=1)
    cols= [1,2,3,4]
    df_ozone = df_ozone.drop(df_ozone.columns[cols], axis=1)
    print(df_ozone.shape)
    print(df_ozone.head())
    return df_ozone


In [34]:
df_ozone_england = unifie_ozone_mean(f_data_ozone_hull, f_data_ozone_norwich, f_data_ozone_wirral, f_data_ozone_london)

(365, 2)
         Date      Ozone
0  01/01/2014  54.285472
1  01/02/2014  48.550482
2  01/03/2014  60.489870
3  01/04/2014  49.537995
4  01/05/2014  51.024612


Após o tratamento dos datasets de Diabetes, Temperatura e Ozono, procedeu-se então à criação do dataset final, que será utilizado nas previsões de Séries Temporais, para isso utilizou-se o método unifie_diabetes_datasets().

In [35]:
def unifie_diabetes_datasets(f_data_diabetes, f_data_temp, f_data_ozone):

    df_diabetes = pd.merge(f_data_diabetes,f_data_temp, on='Date', how='inner')
    df_diabetes = pd.merge(df_diabetes, f_data_ozone, on='Date', how='inner')
    #pd.set_option('display.max_rows', None)
    print(df_diabetes.head())

    return df_diabetes

In [36]:
df_diabetes = unifie_diabetes_datasets(f_data_diabetes, df_ozone_england, df_temp_england)

         Date  All_Causes  Diabetes      Ozone  Temperature
0  01/01/2014        1418        15  54.285472     6.966667
1  02/01/2014        1485        17  62.274416     7.200000
2  03/01/2014        1456        25  34.028751     7.600000
3  04/01/2014        1448        13  35.512845     6.366667
4  05/01/2014        1411        20  62.606071     5.200000


Ainda antes de passar para ficheiro CSV, a data dos registos foi tornada como row.index.

In [37]:
df_diabetes["Date"] = pd.to_datetime(df_diabetes["Date"])
df_diabetes = df_diabetes.set_index('Date')
print(df_diabetes.head())

            All_Causes  Diabetes      Ozone  Temperature
Date                                                    
2014-01-01        1418        15  54.285472     6.966667
2014-02-01        1485        17  62.274416     7.200000
2014-03-01        1456        25  34.028751     7.600000
2014-04-01        1448        13  35.512845     6.366667
2014-05-01        1411        20  62.606071     5.200000


Finalmente, através do método to_csv_file, foi criado o ficheiro CSV do datset final.

In [38]:
to_csv_file(df_diabetes, "daily_diabetes.csv")